# Шкарбаненко Михаил Б05-907

# Задача 6 (6.1)

Рассмотрим задачу предсказание числа заболевших некоторой болезнью от некоторых экологических анализов (см. [файл](https://github.com/andriygav/PSAD/blob/master/labs/lab1/data/sick.csv)). Гарантируется, что предсказание описывается линейной моделью.

Так как проведение анализов не является бесплатным, то стоит вопрос о том какие из анализов являются лишними (на уровне значимости \alpha=0.05) для предсказания линейной модели.

Требуется:

Записать задачу формально;
Провести отбор признаков линейной модели.
Все выкладки должны быть сделаны аналитически, без использования компьютера. (допускается использование компютера для подстановвки численых значений в финальную формулу

# Решение

Задача формально:

Пусть имеется набор данных, состоящий из n наблюдений. Каждое наблюдение содержит информацию об экологических анализах и числе заболевших некоторой болезнью. Предсказание числа заболевших описывается линейной моделью:

$$y_i = \beta_0 + \beta_1 x_{i,1} + \beta_2 x_{i,2} + ... + \beta_p x_{i,k} + \epsilon_i,\ i = 1, 2, ..., n$$

где $y_i$ - число заболевших, $x_{i,1}, x_{i,2}, ..., x_{i,k}$ - значения экологических анализов, $\beta_0, \beta_1, \beta_2, ..., \beta_k$ - коэффициенты линейной модели, $\epsilon_i$ - случайная ошибка. Требуется провести отбор признаков линейной модели на уровне значимости $\alpha=0.05$.

Используем критерий Стьюдента. $H_0: \beta_j = 0$ против альтернативной гипотезы $H_1: \beta_j \neq 0$ для $j = 1, 2, ..., k$. Тогда нулевая гипотеза $H_0$ означает, что $j$-ый признак не влияет на предсказание. Альтернативная гипотеза $H_1$ означает, что $j$-ый признак влияет на предсказание.

Статистика:

$$t_{\beta_j} = \frac{\hat{\beta}_j}{\sqrt{\frac{RSS}{n-k-1}(X^TX)^{-1}_{jj}}}$$

где $\hat{\beta}_j$ - МНК оценка коэффициента регрессии $\beta_j$.

Если $|t_{\beta_j}| > t_{1 - \frac{\alpha}{2}, N-k-1}$, то гипотеза $H_0$ отвергается на уровне значимости $\alpha$, и признак $x_{j}$ влияет на результат.

## Численный расчет

Был добавлен признак $x_0$, чтобы учесть байес. Все вычисления произведены без использования готовых решений.

In [217]:
import pandas as pd
from collections import Counter
import itertools
from matplotlib import pyplot as plt
import numpy as np
from sklearn.feature_selection import f_regression, r_regression, mutual_info_regression
from sklearn.linear_model import LinearRegression
import warnings
import seaborn as sn
warnings.filterwarnings('ignore')
from scipy.stats import t
from sklearn.metrics import r2_score

In [218]:
data = pd.read_csv('https://raw.githubusercontent.com/andriygav/PSAD/master/labs/lab1/data/sick.csv')
X, y = data.loc[:, data.columns != 'Y'].values, data['Y'].values
X = np.concatenate((np.ones(30)[:, np.newaxis], X), axis=1)
print(data.shape, X.shape, y.shape)

(30, 11) (30, 11) (30,)


In [219]:
data.sample(5)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,Y
11,-0.7,-0.2,1.8,1.8,-0.6,-0.5,0.1,1.0,-1.8,0.1,14.0
21,-0.2,1.9,-0.3,-1.3,0.1,1.2,0.3,-0.9,2.3,0.2,1.0
28,-0.3,-0.8,0.9,-1.6,0.2,-0.3,-1.2,1.8,-0.4,1.0,4.0
22,-0.7,1.3,-0.1,0.8,0.4,1.8,-0.4,-0.9,-1.7,-0.2,1.0
24,0.1,1.5,-0.4,-0.4,-0.7,0.7,0.3,0.3,-0.2,0.5,1.0


In [220]:
b_est = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
y_est = X.dot(b_est)
n, k = 30, 11
RSS = np.square(y - y_est).sum()
std = np.sqrt([RSS * np.linalg.inv(X.T.dot(X))[i][i] / (n-k-1) for i in range(k)])
alpha = 0.05
quantile = t.ppf(1-alpha/2, n-k-1)
t_statistic = b_est / std
mask = []
for i in range(k):
    if abs(t_statistic[i]) > quantile:
        mask.append(True)
        print('Отвергаем гипотезу H0 для {}-го признака. Признак x{} не может быть исключен.'.format(i, i)) 
    else:
        mask.append(False)
        print('Данные не противоречат гипотезе H0 для {}-го признака. Признак x{} может быть исключен.'.format(i, i)) 

Отвергаем гипотезу H0 для 0-го признака. Признак x0 не может быть исключен.
Отвергаем гипотезу H0 для 1-го признака. Признак x1 не может быть исключен.
Данные не противоречат гипотезе H0 для 2-го признака. Признак x2 может быть исключен.
Отвергаем гипотезу H0 для 3-го признака. Признак x3 не может быть исключен.
Данные не противоречат гипотезе H0 для 4-го признака. Признак x4 может быть исключен.
Данные не противоречат гипотезе H0 для 5-го признака. Признак x5 может быть исключен.
Данные не противоречат гипотезе H0 для 6-го признака. Признак x6 может быть исключен.
Данные не противоречат гипотезе H0 для 7-го признака. Признак x7 может быть исключен.
Данные не противоречат гипотезе H0 для 8-го признака. Признак x8 может быть исключен.
Данные не противоречат гипотезе H0 для 9-го признака. Признак x9 может быть исключен.
Данные не противоречат гипотезе H0 для 10-го признака. Признак x10 может быть исключен.


In [221]:
print('Коэффициент детерминации до отбрасывания признаков {}'.format(round(r2_score(y, y_est), 2)))
X = X[:, mask]
b_est = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
y_est = X.dot(b_est)
print('Коэффициент детерминации после отбрасывания признаков {}'.format(round(r2_score(y, y_est), 2)))

Коэффициент детерминации до отбрасывания признаков 0.74
Коэффициент детерминации после отбрасывания признаков 0.69


## Вывод

Получилось значительно сжать данные, выбросив большую часть признаков, которые, согласно t-тесту, не связаны с целевой переменной. R2-score упал незначительно.